In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
print(X.shape)
print(y.shape)
n_samples, n_features = X.shape

(569, 30)
(569,)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [23]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [24]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [25]:
model = LogisticRegression(n_features)

In [26]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [27]:
epochs = 100
for epoch in range(epochs):
    
    y_pred = model(X_train)
    
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        [w,b] = model.parameters()
        print(f"Epoch {epoch + 1}: w = {w[0][0]:.3f}, b = {b[0]:.3f}, loss = {loss:.8f}")

Epoch 10: w = -0.203, b = -0.039, loss = 0.50433022
Epoch 20: w = -0.223, b = -0.025, loss = 0.43663275
Epoch 30: w = -0.240, b = -0.013, loss = 0.38967738
Epoch 40: w = -0.255, b = -0.001, loss = 0.35502133
Epoch 50: w = -0.268, b = 0.010, loss = 0.32824203
Epoch 60: w = -0.280, b = 0.020, loss = 0.30681783
Epoch 70: w = -0.290, b = 0.030, loss = 0.28920931
Epoch 80: w = -0.300, b = 0.039, loss = 0.27442354
Epoch 90: w = -0.309, b = 0.047, loss = 0.26179099
Epoch 100: w = -0.317, b = 0.055, loss = 0.25084308


In [28]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test).sum()/ float(y_test.shape[0])
    print(f"accuracy = {acc:.4f}")

accuracy = 0.9298
